### TAO remote client - Purpose built models

Transfer learning is the process of transferring learned features from one application to another. It is a commonly used training technique where you use a model trained on one task and re-train to use it on a different task. Train Adapt Optimize (TAO) Toolkit  is a simple and easy-to-use Python based AI toolkit for taking purpose-built AI models and customizing them with users' own data.

![image](https://d29g4g2dyqv443.cloudfront.net/sites/default/files/akamai/TAO/tlt-tao-toolkit-bring-your-own-model-diagram.png)


### The workflow in a nutshell

- Creating a dataset
- Upload dataset to the service
- Running dataset convert (for specific models)
- Getting a PTM from NGC
- Model Actions
    - Train (Normal/AutoML)
    - Evaluate
    - Prune, retrain (for specific models)
    - Export
    - TAO-Deploy (for specific models)
    - Inference on TAO
    - Inference on TRT (for specific models)

### Table of contents

1. [Install TAO remote client ](#head-1)
1. [Set the remote service base URL](#head-2)
1. [Access the shared volume](#head-3)
1. [Create the datasets](#head-4)
1. [List datasets](#head-5)
1. [Provide and customize dataset convert specs for train dataset](#head-6)
1. [Run dataset convert for train dataset](#head-7)
1. [Provide and customize dataset convert specs for val dataset](#head-8)
1. [Run dataset convert for val dataset](#head-9)
1. [Create an experiment](#head-10)
1. [Find pretrained model](#head-11)
1. [Customize model metadata](#head-12)
1. [View hyperparameters that are enabled for AutoML by default](#head-13)
1. [Set AutoML related configurations](#head-14)
1. [Provide train specs](#head-15)
1. [Run train](#head-16)
1. [View checkpoint files](#head-17)
1. [Provide evaluate specs](#head-18)
1. [Run evaluate](#head-19)
1. [Provide prune specs](#head-20) (for specific models)
1. [Run prune](#head-21) (for specific models)
1. [Provide retrain specs](#head-22) (for specific models)
1. [Run retrain](#head-23) (for specific models)
1. [Run evaluate on retrain](#head-24) (for specific models)
1. [Provide export specs](#head-25)
1. [Run export](#head-26)
1. [Provide trt engine generation specs](#head-27) (for specific models)
1. [Run TRT Engine generation using TAO-Deploy](#head-28) (for specific models)
1. [Provide TAO inference specs](#head-29)
1. [Run TAO inference](#head-30)
1. [Provide TRT inference specs](#head-31) (for specific models)
1. [Run TRT inference](#head-32) (for specific models)
1. [Delete experiment](#head-33)
1. [Delete datasets](#head-34)
1. [Unmount shared volume](#head-35)
1. [Uninstall TAO Remote Client](#head-36)

### Requirements
Please find the server requirements [here](https://docs.nvidia.com/tao/tao-toolkit/text/tao_toolkit_api/api_setup.html#)

In [ ]:
import os
import glob
import subprocess
import json
import time
import ast
import shutil
from IPython.display import clear_output

In [ ]:
namespace = 'default'

### FIXME

1. Assign a model_name in FIXME 1

    1.1 Assign model type for action_recognition/pose_classification in FIXME 1.1

    1.2 Assign platform for action_recognition in FIXME 1.2
    
    1.3 Assign model input type for action_recognition in FIXME 1.3
1. Assign a workdir in FIXME 2
1. Assign the ip_address and port_number in FIXME 3 ([info](https://docs.nvidia.com/tao/tao-toolkit/text/tao_toolkit_api/api_rest_api.html))
1. Assign the ngc_api_key variable in FIXME 4
1. (Optional) Enable AutoML if needed in FIXME 5
1. (Optional) Choose between bayesian and hyperband automl_algorithm in FIXME 6 (If automl was enabled in FIXME5)
1. Choose to download jobs or not in FIXME 7
1. Choose between default and custom dataset in FIXME 8
1. Assign path of DATA_DIR in FIXME 9

In [ ]:
# Define model_name workspaces and other variables
# Available models (#FIXME 1):
# 1. action_recognition - https://docs.nvidia.com/tao/tao-toolkit/text/action_recognition_net.html
# 2. bpnet - https://docs.nvidia.com/tao/tao-toolkit/text/bodypose_estimation/bodyposenet.html
# 3. fpenet - https://docs.nvidia.com/tao/tao-toolkit/text/facial_landmarks_estimation/facial_landmarks_estimation.html
# 4. lprnet - https://docs.nvidia.com/tao/tao-toolkit/text/character_recognition/index.html
# 5. ml_recog - https://docs.nvidia.com/tao/tao-toolkit/text/ml_recog/index.html
# 6. ocdnet - https://docs.nvidia.com/tao/tao-toolkit/text/ocdnet/index.html
# 7. ocrnet - https://docs.nvidia.com/tao/tao-toolkit/text/ocrnet/index.html
# 8. optical_inspection - https://docs.nvidia.com/tao/tao-toolkit/text/optical_inspection/index.html
# 9. pose_classification - https://docs.nvidia.com/tao/tao-toolkit/text/pose_classification/index.html
# 10. pointpillars - https://docs.nvidia.com/tao/tao-toolkit/text/point_cloud/pointpillars.html
# 11. re_identification - https://docs.nvidia.com/tao/tao-toolkit/text/re_identification/index.html
# 12. centerpose
# 13. visual_changenet_segment - https://docs.nvidia.com/tao/tao-toolkit/text/visual_changenet/index.html
# 14. visual_changenet_classify - https://docs.nvidia.com/tao/tao-toolkit/text/visual_changenet/index.html

model_name = "action_recognition" # FIXME1 (Add the model name from the above mentioned list)

In [ ]:
if model_name in ("action_recognition","fpenet","lprnet","pose_classification"):
    # FIXME1.1 - model_type - string
        # action_recognition: rgb/of/joint;
        # fpenet: 10/80 (value represents the number of keypoints)
        # lprnet: us/ch (us for United States, ch for China)
        # pose_classification: kinetics/nvidia
    model_type = "rgb" # FIXME1.1 action_recognition: rgb/of/joint; fpenet: 10/80 (refers to number of keypoints); pose_classification: kinetics/nvidia

    if model_name == "action_recognition":
        if model_type not in ("rgb","of","joint"):
            raise Exception("Choose one of rgb/of/joint for action recognition model_type")
    elif model_name == "fpenet":
        if model_type not in ("10","80"):
            raise Exception("Choose one of 10/80 for FPENET model_type")
    elif model_name == "lprnet":
        if model_type not in ("us","ch"):
            raise Exception("Choose one of us/ch for LPRNET model_type")
    elif model_name == "pose_classification":
        if model_type not in ("kinetics","nvidia"):
            raise Exception("Choose one of kinetics/nvidia for pose classification model_type")

    if model_name == "action_recognition":
        platform = "a100" # FIXME1.2 a100/xavier - valid only for model_type that is not rgb
        model_input_type = "3d" # FIXME1.3 3d/2d

### Install TAO remote client <a class="anchor" id="head-1"></a>

In [ ]:
# SKIP this step IF you have already installed the TAO-Client wheel.
! pip3 install nvidia-transfer-learning-client


In [ ]:
# View the version of the TAO-Client
! nvtl --version

### Set the remote service base URL and Token <a class="anchor" id="head-2"></a>

In [ ]:
# Define the node_addr and port number
workdir = "workdir_purpose_built_models" # FIXME2
host_url = "http://<ip_address>:<port_number>" # FIXME3 example: https://10.137.149.22:32334
# In host machine, node ip_address and port number can be obtained as follows,
# ip_address: hostname -i
# port_number: kubectl get service ingress-nginx-controller -o jsonpath='{.spec.ports[0].nodePort}'

ngc_api_key = "<ngc_api_key>" # FIXME4 example: (Add NGC API key)

In [ ]:
automl_enabled = False # FIXME5 set to True if you want to run automl for the model chosen in the previous cell
automl_algorithm="bayesian" # FIXME6 example: bayesian/hyperband
# FIXME7 Defaulted to False as downloading jobs from service to your machine takes time
# Set to True if you want to download jobs where examples have been provided like for train, export, inference.
download_jobs = False

In [ ]:
%env BASE_URL={host_url}/{namespace}/api/v1

# Exchange NGC_API_KEY for JWT
identity = json.loads(subprocess.getoutput(f"nvtl login --ngc-api-key {ngc_api_key}"))

%env USER={identity['user_id']}
%env TOKEN={identity['token']}

In [ ]:
# Creating workdir
workdir = os.path.abspath(workdir)
if not os.path.isdir(workdir):
    os.makedirs(workdir)

### Function to parse logs <a class="anchor" id="head-1.1"></a>

In [ ]:
def my_tail(model_name_cli, id, job_id, job_type, workdir):
	status = None
	while True:
		time.sleep(10)
		clear_output(wait=True)
		log_file_path = subprocess.getoutput(f"nvtl {model_name_cli} get-log-file --id {id} --job {job_id} --job_type {job_type} --workdir {workdir}")
		if not os.path.exists(log_file_path):
			continue
		with open(log_file_path, 'rb') as log_file:
			log_contents = log_file.read()
		log_content_lines = log_contents.decode("utf-8").split("\n")        
		for line in log_content_lines:
			print(line.strip())
			if line.strip() == "Error EOF":
				status = "Error"
				break
			elif line.strip() == "Done EOF":
				status = "Done"
				break
		if status is not None:
			break
	return status

### Function to split tar files <a class="anchor" id="head-1.1"></a>

In [ ]:
import os
import tarfile

def split_tar_file(input_tar_path, output_dir, max_split_size=0.2*1024*1024*1024):
	os.makedirs(output_dir, exist_ok=True)
	
	with tarfile.open(input_tar_path, 'r') as original_tar:
		members = original_tar.getmembers()
		current_split_size = 0
		current_split_number = 0
		current_split_name = os.path.join(output_dir, f'smaller_file_{current_split_number}.tar')
		
		with tarfile.open(current_split_name, 'w') as split_tar:
			for member in members:
				if current_split_size + member.size <= max_split_size:
					split_tar.addfile(member, original_tar.extractfile(member))
					current_split_size += member.size
				else:
					split_tar.close()
					current_split_number += 1
					current_split_name = os.path.join(output_dir, f'smaller_file_{current_split_number}.tar')
					current_split_size = 0
					split_tar = tarfile.open(current_split_name, 'w')  # Open a new split tar archive
					split_tar.addfile(member, original_tar.extractfile(member))
					current_split_size += member.size

### Set dataset type, format <a class="anchor" id="head-1.1"></a>

**Action Recognition:** We will be using the HMDB51 [dataset](https://serre-lab.clps.brown.edu/resource/hmdb-a-large-human-motion-database/) for the tutorial. (We choose catch/smile for this tutorial):

**BPNET:** We will be using the `COCO dataset` for Instance segmentation - MaskRCNN. `download_coco.sh` script from dataset prepare will be used to download and unzip the coco2017 dataset from [here](https://cocodataset.org/#download)

**FPENET:** We will be using `AFW dataset`. Download it from [here](https://ibug.doc.ic.ac.uk/download/annotations/afw.zip/) and place it in $DATA_DIR.

**LPRNET**: We will be using the `OpenALPR benchmark dataset` for the tutorial. The following script will download the dataset automatically and convert it to the format used by TAO.  

**MLRecogNet** We will be using the `Retail Product Checkout Dataset` for the tutorial. Downdload the datsaet from [here](https://www.kaggle.com/datasets/diyer22/retail-product-checkout-dataset) and place it under $DATA_DIR/metric_learning_recognition

**OCDNET**: We will be using the ICDAR2015 dataset for the ocdnet tutorial. Please access the dataset [here](https://rrc.cvc.uab.es/?ch=4&com=tasks) to register and download the data from Task 4.1: Text Localization. Unzip the files to DATA_DIR

**OCRNET**: We will be using the ICDAR15 word recognition dataset for the tutorial. To find more details please visit [here](
https://rrc.cvc.uab.es/?ch=4&com=tasks). Please download the ICDAR15 word recognition train dataset and test_dataset [here](https://rrc.cvc.uab.es/?ch=4&com=downloads) to DATA_DIR.

**Pointpillars:** We will be using the `kitti object detection dataset` for this example. To find more details, please visit [here](http://www.cvlibs.net/datasets/kitti/eval_object.php?obj_benchmark=2d)

**Pose Classification:** We will be using the Kinetics dataset from [Deepmind](https://deepmind.com/research/open-source/kinetics) or NVIDIA created dataset. For kinetics based dataset set model_type as `kinetics` and for nvidia based dataset set model_type as `nvidia`

**re_identification:** We will be using the [Market-1501](https://zheng-lab.cecs.anu.edu.au/Project/project_reid.html) dataset. Download the dataset [here](https://drive.google.com/file/d/1TwkgQcIa_EgRjVMPSbyEKtcfljqURrzi/view?usp=sharing) and extract it.

**Optical Inspection:** Bring your own dataset according to the format described [here](https://docs.nvidia.com/tao/tao-toolkit/text/data_annotation_format.html#optical_inspection-format). 

**Visual ChangeNet-Classification:** Bring your own dataset according to the format described [here](https://docs.nvidia.com/tao/tao-toolkit/text/data_annotation_format.html#optical_inspection-format). 

**Visual ChangeNet-Segmentation:** We will be using the [Market-1501](https://zheng-lab.cecs.anu.edu.au/Project/project_reid.html) dataset. Download the dataset [here](https://www.dropbox.com/s/18fb5jo0npu5evm/LEVIR-CD256.zip) and extract it. 

**CenterPose:** We will be using [Google Objectron](https://github.com/google-research-datasets/Objectron) dataset. The following script will download and preprocess the dataset the dataset automatically.

In [ ]:
if model_name == "lprnet":
    ds_type = "character_recognition"
    ds_format = "lprnet"
elif model_name in ("visual_changenet_classify", "visual_changenet_segment"):
    ds_type = "visual_changenet"
    ds_format = model_name
    model_name = 'visual_changenet'
else:
    ds_type = model_name
    ds_format = "default"

In [ ]:
dataset_to_be_used = "default" #FIXME8 #default/custom; default for the dataset used in this tutorial notebook; custom for a different dataset
DATA_DIR = os.path.abspath(model_name) # FIXME9 (set absolute path of the data_directory)
os.environ['DATA_DIR']= DATA_DIR
!mkdir -p {DATA_DIR}
job_map = {}

### Dataset download and pre-processing <a class="anchor" id="head-1"></a>

In [ ]:
if dataset_to_be_used == "default":
    if model_name == "action_recognition":
        !sudo apt-get update -y && sudo apt-get install unrar-free -y
        !wget -P $DATA_DIR http://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/hmdb51_org.rar
        assert os.path.exists(f"{DATA_DIR}/hmdb51_org.rar")
        !mkdir -p $DATA_DIR/videos && unrar x -o+ $DATA_DIR/hmdb51_org.rar $DATA_DIR/videos
        !mkdir -p $DATA_DIR/raw_data
        !unrar x -o+ $DATA_DIR/videos/catch.rar $DATA_DIR/raw_data
        !unrar x -o+ $DATA_DIR/videos/smile.rar $DATA_DIR/raw_data
        assert os.path.exists(f"{DATA_DIR}/raw_data/catch")
        assert os.path.exists(f"{DATA_DIR}/raw_data/smile")
    elif model_name == "bpnet":
        !bash ../dataset_prepare/coco/download_coco.sh $DATA_DIR
        # Remove existing data
        !rm -rf $DATA_DIR/train2017/images
        !rm -rf $DATA_DIR/val2017/images
        # Rearrange data in the required format
        !mv $DATA_DIR/raw-data/* $DATA_DIR/
        !cp ../dataset_prepare/bpnet/* $DATA_DIR/
        assert os.path.exists(f"{DATA_DIR}/train2017")
        assert os.path.exists(f"{DATA_DIR}/val2017")
        assert os.path.exists(f"{DATA_DIR}/annotations")
        assert os.path.exists(f"{DATA_DIR}/bpnet_18joints.json")
        assert os.path.exists(f"{DATA_DIR}/coco_spec.json")
        assert os.path.exists(f"{DATA_DIR}/infer_spec.yaml")
    elif model_name == "fpenet":
        assert os.path.exists(f"{DATA_DIR}/afw.zip")
        !mkdir $DATA_DIR/data
        !unzip -uq $DATA_DIR/afw.zip -d $DATA_DIR/data/afw
        !cp ../dataset_prepare/fpenet/data.json $DATA_DIR/
        assert os.path.exists(f"{DATA_DIR}/data/afw")
        assert os.path.exists(f"{DATA_DIR}/data.json")
    elif model_name == "lprnet":
        !python3 -m pip install --upgrade pip
        !python3 -m pip install "opencv-python>=3.4.0.12,<=4.5.5.64"
        !bash ../dataset_prepare/lprnet/download_and_prepare_data.sh $DATA_DIR
        assert os.path.exists(f"{DATA_DIR}/train/image")
        assert os.path.exists(f"{DATA_DIR}/train/label")
        assert os.path.exists(f"{DATA_DIR}/val/image")
        assert os.path.exists(f"{DATA_DIR}/val/label")
    elif model_name == "ml_recog":
        assert os.path.exists(f"{DATA_DIR}/metric_learning_recognition/retail-product-checkout-dataset.zip")
        !unzip -uq $DATA_DIR/metric_learning_recognition/retail-product-checkout-dataset.zip -d $DATA_DIR/metric_learning_recognition
    elif model_name == "ocdnet":
        assert(os.path.exists(f"{DATA_DIR}/train/img"))
        assert(os.path.exists(f"{DATA_DIR}/train/gt"))
        assert(os.path.exists(f"{DATA_DIR}/test/img"))
        assert(os.path.exists(f"{DATA_DIR}/test/gt"))
    elif model_name == "ocrnet":
        !mkdir -p $DATA_DIR/train && rm -rf $DATA_DIR/train/*
        !mkdir -p $DATA_DIR/test && rm -rf $DATA_DIR/test/*
        !unzip -u $DATA_DIR/ch4_test_word_images_gt.zip -d $DATA_DIR/test
        !cp $DATA_DIR/Challenge4_Test_Task3_GT.txt -d $DATA_DIR/test
        !unzip -u $DATA_DIR/ch4_training_word_images_gt.zip -d $DATA_DIR/train    
        assert os.path.exists(f"{DATA_DIR}/ch4_test_word_images_gt.zip")
        assert os.path.exists(f"{DATA_DIR}/Challenge4_Test_Task3_GT.txt")
        assert os.path.exists(f"{DATA_DIR}/ch4_training_word_images_gt.zip")
    elif model_name == "optical_inspection" or (model_name == "visual_changenet" and ds_format == "visual_changenet_classify"):
        assert os.path.exists(f"{DATA_DIR}/train/images")
        assert os.path.exists(f"{DATA_DIR}/train/dataset.csv")
        assert os.path.exists(f"{DATA_DIR}/val/images")
        assert os.path.exists(f"{DATA_DIR}/val/dataset.csv")
        assert os.path.exists(f"{DATA_DIR}/test/images")
        assert os.path.exists(f"{DATA_DIR}/test/dataset.csv")
    elif model_name == "visual_changenet" and ds_format == "visual_changenet_segment":
        #Download the data
        URL_DATASET = "https://www.dropbox.com/s/18fb5jo0npu5evm/LEVIR-CD256.zip"
        os.environ["URL_DATASET"]=URL_DATASET
        !if [ ! -f $DATA_DIR/LEVIR-CD256.zip ]; then wget $URL_DATASET -O $DATA_DIR/LEVIR-CD-256.zip; else echo "image archive already downloaded"; fi 
        # Check the dataset is present
        !mkdir -p $DATA_DIR
        !if [ ! -f $DATA_DIR/LEVIR-CD-256.zip ]; then echo 'Dataset zip file not found, please download.'; else echo 'Found Dataset zip file.';fi
        # unpack 
        !unzip -u $DATA_DIR/LEVIR-CD-256.zip -d $DATA_DIR
        assert os.path.exists(f"{DATA_DIR}/LEVIR-CD256/A")
        assert os.path.exists(f"{DATA_DIR}/LEVIR-CD256/B")
        assert os.path.exists(f"{DATA_DIR}/LEVIR-CD256/label")
        assert os.path.exists(f"{DATA_DIR}/LEVIR-CD256/list/train.txt")
        assert os.path.exists(f"{DATA_DIR}/LEVIR-CD256/list/val.txt")
        assert os.path.exists(f"{DATA_DIR}/LEVIR-CD256/list/test.txt")
    elif model_name == "pointpillars":
        !unzip -u $DATA_DIR/data_object_image_2.zip -d $DATA_DIR
        !unzip -u $DATA_DIR/data_object_label_2.zip -d $DATA_DIR
        !unzip -u $DATA_DIR/data_object_velodyne.zip -d $DATA_DIR
        !unzip -u $DATA_DIR/data_object_calib.zip -d $DATA_DIR
        assert os.path.exists(f"{DATA_DIR}/training/image_2")
        assert os.path.exists(f"{DATA_DIR}/training/label_2")
        assert os.path.exists(f"{DATA_DIR}/training/velodyne")
        assert os.path.exists(f"{DATA_DIR}/training/calib")
    elif model_name == "pose_classification":
        !pip3 install -U gdown
        if model_type == "kinetics":
            !gdown https://drive.google.com/uc?id=1dmzCRQsFXJ18BlXj1G9sbDnsclXIdDdR -O $DATA_DIR/st-gcn-processed-data.zip
            !unzip $DATA_DIR/st-gcn-processed-data.zip -d $DATA_DIR
            !mv $DATA_DIR/data/Kinetics/kinetics-skeleton $DATA_DIR/kinetics
            !rm -r $DATA_DIR/data
            !rm $DATA_DIR/st-gcn-processed-data.zip
            assert os.path.exists(f"{DATA_DIR}/kinetics")
        elif model_type == "nvidia":
            !gdown https://drive.google.com/uc?id=1GhSt53-7MlFfauEZ2YkuzOaZVNIGo_c- -O $DATA_DIR/data_3dbp_nvidia.zip
            !mkdir -p $DATA_DIR/nvidia
            !unzip $DATA_DIR/data_3dbp_nvidia.zip -d $DATA_DIR/nvidia
            !rm $DATA_DIR/data_3dbp_nvidia.zip
            assert os.path.exists(f"{DATA_DIR}/nvidia")
            assert os.path.exists(f"{DATA_DIR}/{model_type}/train_data.npy") and os.path.exists(f"{DATA_DIR}/{model_type}/train_label.pkl") and os.path.exists(f"{DATA_DIR}/{model_type}/val_data.npy") and os.path.exists(f"{DATA_DIR}/{model_type}/val_label.pkl")
    elif model_name == "re_identification":
        !pip3 install -U gdown
        !gdown https://drive.google.com/uc?id=0B8-rUzbwVRk0c054eEozWG9COHM -O $DATA_DIR/market1501.zip
        !unzip -u $DATA_DIR/market1501.zip -d $DATA_DIR
        !rm -rf $DATA_DIR/market1501
        !mv $DATA_DIR/Market-1501-v15.09.15 $DATA_DIR/market1501
        !rm $DATA_DIR/market1501.zip
        assert os.path.exists(f"{DATA_DIR}/market1501")

In [ ]:
if model_name in ("lprnet","ocdnet","ocrnet", "optical_inspection") or ds_format == "visual_changenet_classify":
    eval_dataset_path = f"{DATA_DIR}/purpose_built_models_val.tar.gz"
if model_name in ("lprnet", "optical_inspection") or ds_format == "visual_changenet_classify":
    test_dataset_path = f"{DATA_DIR}/purpose_built_models_test.tar.gz"
train_dataset_path = f"{DATA_DIR}/purpose_built_models_train.tar.gz"

In [ ]:
train_dataset_id = subprocess.getoutput(f"nvtl {model_name} dataset-create --dataset_type {ds_type} --dataset_format {ds_format}")
print(train_dataset_id)

In [ ]:
if dataset_to_be_used == "default":
    USER_EXPERIMENT_DIR = os.path.join("/shared/users",os.environ["USER"],"datasets",train_dataset_id)
    if model_name == "action_recognition":
        !python3 -m pip install opencv-python numpy
        # For rgb action recognition
        !if [ -d tao_toolkit_recipes ]; then rm -rf tao_toolkit_recipes; fi
        !git clone https://github.com/NVIDIA-AI-IOT/tao_toolkit_recipes
        assert os.path.exists("tao_toolkit_recipes")
        !cd tao_toolkit_recipes/tao_action_recognition/data_generation/ && bash ./preprocess_HMDB_RGB.sh $DATA_DIR/raw_data $DATA_DIR/processed_data 

        # For optical flow, comment the above 3 lines and uncomment the below (Note: for generating optical flow, a Turing or Ampere above GPU is needed.)
        #!echo <passwd> | sudo -S apt install -y libfreeimage-dev
        #!cp ../dataset_prepare/action_recognition/AppOFCuda tao_toolkit_recipes/tao_action_recognition/data_generation/
        #!cd tao_toolkit_recipes/tao_action_recognition/data_generation/ && bash ./preprocess_HMDB.sh $DATA_DIR/raw_data $DATA_DIR/processed_data

        # download the split files and unrar
        !wget -P $DATA_DIR http://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/test_train_splits.rar
        assert os.path.exists(f"{DATA_DIR}/test_train_splits.rar")
        !mkdir -p $DATA_DIR/splits && unrar x -o+ $DATA_DIR/test_train_splits.rar $DATA_DIR/splits
        assert os.path.exists(f"{DATA_DIR}/splits")
        # run split_HMDB to generate training split
        !if [ -d $DATA_DIR/train ]; then rm -rf $DATA_DIR/train $DATA_DIR/test; fi
        !cd tao_toolkit_recipes/tao_action_recognition/data_generation/ && python3 ./split_dataset.py $DATA_DIR/processed_data $DATA_DIR/splits/testTrainMulti_7030_splits $DATA_DIR/train  $DATA_DIR/test
        assert os.path.exists(f'{DATA_DIR}/train')
        assert os.path.exists(f'{DATA_DIR}/test')

        if os.path.exists("tao_toolkit_recipes"):
            shutil.rmtree("tao_toolkit_recipes")

        assert not os.path.exists("tao_toolkit_recipes")

    elif model_name == "fpenet":
        !pip3 install numpy opencv-python
        if model_type == "80":
            output_json_path = os.path.join(os.environ['DATA_DIR'], 'data/afw/afw.json')
        elif model_type == "10":
            output_json_path = os.path.join(os.environ['DATA_DIR'], 'data/afw_10/afw_10.json')
        !python3 ../dataset_prepare/fpenet/data_utils.py --afw_data_path $DATA_DIR/data/afw --output_json_path $output_json_path --afw_image_save_path $DATA_DIR/data/afw --num_key_points $model_type --container_root_path $USER_EXPERIMENT_DIR
        assert os.path.exists(output_json_path)
        with open(output_json_path, encoding='utf-8') as afw_json_file:
            afw_json = json.load(afw_json_file)
            assert afw_json

    elif model_name == "lprnet":
        character_file_link = "https://api.ngc.nvidia.com/v2/models/nvidia/tao/lprnet/versions/trainable_v1.0/files/{}_lp_characters.txt".format(model_type)
        !wget -q -O $DATA_DIR/train/characters.txt $character_file_link
        !cp $DATA_DIR/train/characters.txt $DATA_DIR/val/characters.txt
        assert os.path.exists(f"{DATA_DIR}/train/characters.txt")
        assert os.path.exists(f"{DATA_DIR}/val/characters.txt")

    elif model_name == "ml_recog":
        # crops images from detection set and form a classification set
        # splits to reference/train/val/test set
        !sudo apt-get update && sudo apt-get install gcc -y
        !python3 -m pip install opencv-python numpy pycocotools tqdm
        !python3 ../dataset_prepare/metric_learning_recognition/process_retail_product_checkout_dataset.py
        assert os.path.exists(f"{DATA_DIR}/metric_learning_recognition/retail-product-checkout-dataset_classification_demo")
        assert os.path.exists(f"{DATA_DIR}/metric_learning_recognition/retail-product-checkout-dataset_classification_demo/known_classes")
        assert os.path.exists(f"{DATA_DIR}/metric_learning_recognition/retail-product-checkout-dataset_classification_demo/unknown_classes")
        assert os.path.exists(f"{DATA_DIR}/metric_learning_recognition/retail-product-checkout-dataset_classification_demo/known_classes/train")
        assert os.path.exists(f"{DATA_DIR}/metric_learning_recognition/retail-product-checkout-dataset_classification_demo/unknown_classes/train")
        assert os.path.exists(f"{DATA_DIR}/metric_learning_recognition/retail-product-checkout-dataset_classification_demo/known_classes/test")
        assert os.path.exists(f"{DATA_DIR}/metric_learning_recognition/retail-product-checkout-dataset_classification_demo/unknown_classes/test")
        assert os.path.exists(f"{DATA_DIR}/metric_learning_recognition/retail-product-checkout-dataset_classification_demo/known_classes/val")
        assert os.path.exists(f"{DATA_DIR}/metric_learning_recognition/retail-product-checkout-dataset_classification_demo/unknown_classes/val")
        assert os.path.exists(f"{DATA_DIR}/metric_learning_recognition/retail-product-checkout-dataset_classification_demo/known_classes/reference")
        assert os.path.exists(f"{DATA_DIR}/metric_learning_recognition/retail-product-checkout-dataset_classification_demo/unknown_classes/reference")

    elif model_name == "ocrnet":
        orig_train_gt_file=os.path.join(os.getenv("DATA_DIR"), "train", "gt.txt")
        processed_train_gt_file=os.path.join(os.getenv("DATA_DIR"), "train", "gt_new.txt")
        orig_test_gt_file=os.path.join(os.getenv("DATA_DIR"), "test", "Challenge4_Test_Task3_GT.txt")
        processed_test_gt_file=os.path.join(os.getenv("DATA_DIR"), "test", "gt_new.txt")
        !python3 ../dataset_prepare/ocrnet/preprocess_label.py $orig_train_gt_file $processed_train_gt_file
        !python3 ../dataset_prepare/ocrnet/preprocess_label.py $orig_test_gt_file $processed_test_gt_file

    elif model_name == "pointpillars":
        !python3 -m pip install scikit-image numpy
        !mkdir -p $DATA_DIR/train/lidar $DATA_DIR/train/label $DATA_DIR/val/lidar $DATA_DIR/val/label

        # Convert labels from Camera coordinate system to LIDAR coordinate system, etc
        !python3 ../dataset_prepare/pointpillars/gen_lidar_points.py -p $DATA_DIR/training/velodyne \
                                               -c $DATA_DIR/training/calib    \
                                               -i $DATA_DIR/training/image_2  \
                                               -o $DATA_DIR/train/lidar
        assert os.listdir(f"{DATA_DIR}/train/lidar")
        # Drop DontCare class
        !python3 ../dataset_prepare/pointpillars/gen_lidar_labels.py -l $DATA_DIR/training/label_2 \
                                               -c $DATA_DIR/training/calib \
                                               -o $DATA_DIR/train/label
        # train/val split
        !python3 ../dataset_prepare/pointpillars/drop_class.py $DATA_DIR/train/label DontCare
        assert os.listdir(f"{DATA_DIR}/train/label")
        # Change the val set id's if you need a different set of validation images
        !python3 ../dataset_prepare/pointpillars/kitti_split.py ../dataset_prepare/pointpillars/val.txt \
                                          $DATA_DIR/train/lidar \
                                          $DATA_DIR/train/label \
                                          $DATA_DIR/val/lidar \
                                          $DATA_DIR/val/label
        assert os.listdir(f"{DATA_DIR}/val/label")
        assert os.listdir(f"{DATA_DIR}/val/lidar")

    elif model_name == "pose_classification" and model_type == "kinetics":
        !pip3 install numpy
        # select actions
        !python3 ../dataset_prepare/pose_classification/select_subset_actions.py
        assert os.path.exists(f"{DATA_DIR}/{model_type}/train_data.npy") and os.path.exists(f"{DATA_DIR}/{model_type}/train_label.pkl") and os.path.exists(f"{DATA_DIR}/{model_type}/val_data.npy") and os.path.exists(f"{DATA_DIR}/{model_type}/val_label.pkl")

    elif model_name == "re_identification":
        #100 is the number of samples to be present in the subset data - you can choose any number <= total samples in the dataset
        !python3 ../dataset_prepare/re_identification/obtain_subset_data.py 100
        assert os.path.exists(f"{DATA_DIR}/market1501/sample_train")
        assert os.path.exists(f"{DATA_DIR}/market1501/sample_test")
        assert os.path.exists(f"{DATA_DIR}/market1501/sample_query")

    elif model_name == "centerpose":
        # Select the training categories: bike, book, bottle, camera, cereal_box, chair, laptop, shoe
        # Please set the "n" to -1 if you want to run the whole dataset training.
        testing_categories = 'bike'
        !pip3 install numpy opencv-python tqdm scipy==1.9.2 tensorflow==2.14.0
        !python3 ../dataset_prepare/centerpose/prepare_centerpose_dataset.py \
                                            -c $testing_categories \
                                            -n 100
        assert os.path.exists(f"{DATA_DIR}/{testing_categories}/train")
        assert os.path.exists(f"{DATA_DIR}/{testing_categories}/test")
        assert os.path.exists(f"{DATA_DIR}/{testing_categories}/val")

### Tar the datasets <a class="anchor" id="head-1.3"></a>

In [ ]:
if model_name == "action_recognition":
    !tar -C $DATA_DIR -czf $DATA_DIR/purpose_built_models_train.tar.gz train test
elif model_name == "bpnet":
    !tar -C $DATA_DIR -czf $DATA_DIR/purpose_built_models_train.tar.gz train2017 val2017 annotations bpnet_18joints.json  coco_spec.json  infer_spec.yaml
elif model_name == "fpenet":
    !tar -C $DATA_DIR -czf $DATA_DIR/purpose_built_models_train.tar.gz data data.json
elif model_name == "lprnet":
    !tar -C $DATA_DIR/train/ -czf $DATA_DIR/purpose_built_models_train.tar.gz image label characters.txt
    !tar -C $DATA_DIR/val/ -czf $DATA_DIR/purpose_built_models_val.tar.gz image label characters.txt
    !tar -C $DATA_DIR/val/ -czf $DATA_DIR/purpose_built_models_test.tar.gz image
elif model_name == "ml_recog":
    !tar -C $DATA_DIR -czf $DATA_DIR/purpose_built_models_train.tar.gz metric_learning_recognition/retail-product-checkout-dataset_classification_demo/
elif model_name == "ocdnet":
    !tar -C $DATA_DIR -czf $DATA_DIR/purpose_built_models_train.tar.gz train
    !tar -C $DATA_DIR -czf $DATA_DIR/purpose_built_models_val.tar.gz test
elif model_name == "ocrnet":
    !tar -C $DATA_DIR -czf $DATA_DIR/purpose_built_models_train.tar.gz train character_list
    !tar -C $DATA_DIR -czf $DATA_DIR/purpose_built_models_val.tar.gz test character_list
elif model_name == "optical_inspection" or ds_format == "visual_changenet_classify":
    !tar -C $DATA_DIR/train -czf $DATA_DIR/purpose_built_models_train.tar.gz images dataset.csv
    !tar -C $DATA_DIR/val -czf $DATA_DIR/purpose_built_models_val.tar.gz images dataset.csv
    !tar -C $DATA_DIR/test -czf $DATA_DIR/purpose_built_models_test.tar.gz images dataset.csv
elif ds_format == "visual_changenet_segment":
    !tar -C $DATA_DIR/LEVIR-CD256 -czf $DATA_DIR/purpose_built_models_train.tar.gz A B list label
elif model_name == "pointpillars":
    !tar -C $DATA_DIR -czf $DATA_DIR/purpose_built_models_train.tar.gz train val
elif model_name == "pose_classification":
    !tar -C $DATA_DIR -czf $DATA_DIR/purpose_built_models_train.tar.gz $model_type
elif model_name == "re_identification":
    !tar -C $DATA_DIR/market1501 -czf $DATA_DIR/purpose_built_models_train.tar.gz sample_train sample_test sample_query
elif model_name == 'centerpose':
    !tar -C $DATA_DIR/{testing_categories} -czf $DATA_DIR/purpose_built_models_train.tar.gz train val test

### Upload train dataset <a class="anchor" id="head-1.3"></a>

In [ ]:
output_dir = os.path.join(os.path.dirname(os.path.abspath(train_dataset_path)), model_name, "train")
split_tar_file(train_dataset_path, output_dir)
for idx, tar_dataset_path in enumerate(os.listdir(output_dir)):
    print(f"Uploading {idx+1}/{len(os.listdir(output_dir))} tar split")
    upload_train_dataset_message = subprocess.getoutput(f"nvtl {model_name} dataset-upload --id {train_dataset_id} --path {os.path.join(output_dir,tar_dataset_path)}")
    print(upload_train_dataset_message)

### Create and upload val dataset <a class="anchor" id="head-1.3"></a>

In [ ]:
if model_name in ("lprnet","ocdnet","ocrnet","optical_inspection") or ds_format == "visual_changenet_classify":
    eval_dataset_id = subprocess.getoutput(f"nvtl {model_name} dataset-create --dataset_type {ds_type} --dataset_format {ds_format}")
    print(eval_dataset_id)

In [ ]:
if model_name in ("lprnet","ocdnet","ocrnet","optical_inspection") or ds_format == "visual_changenet_classify":
    output_dir = os.path.join(os.path.dirname(os.path.abspath(eval_dataset_path)), model_name, "val")
    split_tar_file(eval_dataset_path, output_dir)
    for idx, tar_dataset_path in enumerate(os.listdir(output_dir)):
        print(f"Uploading {idx+1}/{len(os.listdir(output_dir))} tar split")
        upload_val_dataset_message = subprocess.getoutput(f"nvtl {model_name} dataset-upload --id {eval_dataset_id} --path {os.path.join(output_dir,tar_dataset_path)}")
        print(upload_val_dataset_message)

### Create and upload test dataset <a class="anchor" id="head-1.4"></a>

In [ ]:
if model_name in ("lprnet", "optical_inspection") or ds_format == "visual_changenet_classify":
    if model_name == "lprnet":
        ds_type = "character_recognition"
        ds_format = "raw"
    elif 'visual_changenet' in ds_type:
        ds_type = ds_type
        ds_format = ds_format
    else:
        ds_type = model_name
        ds_format = "default"
    test_dataset_id = subprocess.getoutput(f"nvtl {model_name} dataset-create --dataset_type {ds_type} --dataset_format {ds_format}")
    print(test_dataset_id)

In [ ]:
if model_name in ("lprnet", "optical_inspection") or ds_format == "visual_changenet_classify":
    output_dir = os.path.join(os.path.dirname(os.path.abspath(test_dataset_path)), model_name, "test")
    split_tar_file(test_dataset_path, output_dir)
    for idx, tar_dataset_path in enumerate(os.listdir(output_dir)):
        print(f"Uploading {idx+1}/{len(os.listdir(output_dir))} tar split")
        upload_test_dataset_message = subprocess.getoutput(f"nvtl {model_name} dataset-upload --id {test_dataset_id} --path {os.path.join(output_dir,tar_dataset_path)}")
        print(upload_test_dataset_message)

### List the created datasets <a class="anchor" id="head-5"></a>

In [ ]:
message = subprocess.getoutput(f"nvtl {model_name} list-datasets")
message = ast.literal_eval(message)
for rsp in message:
    rsp_keys = rsp.keys()
    assert "id" in rsp_keys
    assert "type" in rsp_keys
    assert "format" in rsp_keys
    assert "name" in rsp_keys
    print(rsp["id"],"\t",rsp["type"],"\t",rsp["format"],"\t\t",rsp["name"])

### Train Dataset convert Action <a class="anchor" id="head-3"></a>

In [ ]:
# Default train dataset specs
if model_name in ("bpnet", "fpenet", "ocrnet", "pointpillars"):
    train_ds_convert_specs = subprocess.getoutput(f"nvtl {model_name} get-spec --action dataset_convert --id {train_dataset_id} --job_type dataset")
    train_ds_convert_specs = json.loads(train_ds_convert_specs)
    print(json.dumps(train_ds_convert_specs, indent=4))

In [ ]:
# Customize train dataset specs
if model_name in ("bpnet", "fpenet", "ocrnet", "pointpillars"):
    if model_name == "bpnet":
        train_ds_convert_specs["mode"] = "train"
    elif model_name == "fpenet":
        train_ds_convert_specs["num_keypoints"] = int(model_type)
    print(json.dumps(train_ds_convert_specs, indent=4))

In [ ]:
if model_name in ("bpnet", "fpenet", "ocrnet", "pointpillars"):
    job_id = subprocess.getoutput(f"nvtl {model_name} dataset-run-action --id {train_dataset_id}  --action dataset_convert --specs '{json.dumps(train_ds_convert_specs)}'")
    job_map["train_convert_" + model_name] = job_id
    print(job_id)

In [ ]:
# Check status (the file won't exist until the backend Toolkit container is running -- can take several minutes)
if model_name in ("bpnet", "fpenet", "ocrnet", "pointpillars"):
    status = my_tail(model_name, train_dataset_id, job_id, "dataset", workdir)

### Eval Dataset convert Action <a class="anchor" id="head-3"></a>

In [ ]:
# Default val dataset specs
if model_name == "bpnet":
    eval_ds_convert_specs = subprocess.getoutput(f"nvtl {model_name} get-spec --action dataset_convert --id {train_dataset_id} --job_type dataset")
elif model_name == "ocrnet":
    eval_ds_convert_specs = subprocess.getoutput(f"nvtl {model_name} get-spec --action dataset_convert --id {eval_dataset_id} --job_type dataset")

if model_name in ("bpnet", "ocrnet"):
    eval_ds_convert_specs = json.loads(eval_ds_convert_specs)
    print(json.dumps(eval_ds_convert_specs, indent=4))

In [ ]:
# Customize val dataset specs
if model_name in ("bpnet", "ocrnet"):
    # Apply changes to the specs dictionary if necessary
    if model_name == "bpnet":
        eval_ds_convert_specs["mode"] = "test"
    print(json.dumps(eval_ds_convert_specs, indent=4))

In [ ]:
if model_name in ("bpnet", "ocrnet"):
    train_convert_job_id = job_map["train_convert_" + model_name]
    if model_name == "bpnet":
        job_id = subprocess.getoutput(f"nvtl {model_name} dataset-run-action --id {train_dataset_id} --action dataset_convert --parent_job_id {train_convert_job_id} --specs '{json.dumps(eval_ds_convert_specs)}'")
    elif model_name == "ocrnet":
        job_id = subprocess.getoutput(f"nvtl {model_name} dataset-run-action --id {eval_dataset_id} --action dataset_convert --parent_job_id {train_convert_job_id} --specs '{json.dumps(eval_ds_convert_specs)}'")
    job_map["eval_convert_" + model_name] = job_id
    print(job_id)

In [ ]:
# Check status (the file won't exist until the backend Toolkit container is running -- can take several minutes)
if model_name in ("bpnet", "ocrnet"):
    if model_name == "bpnet":
        status = my_tail(model_name, train_dataset_id, job_id, "dataset", workdir)
    elif model_name == "ocrnet":
        status = my_tail(model_name, eval_dataset_id, job_id, "dataset", workdir)

### Create an experiment <a class="anchor" id="head-8"></a>

In [ ]:
if model_name in ("action_recognition", "pose_classification", "ml_recog", "ocrnet", "ocdnet", "optical_inspection", "re_identification"):
    encode_key = "nvidia_tao"
elif model_name == "pointpillars":
    encode_key = "tlt_encode"
else:
    encode_key = "nvidia_tlt"

network_arch = model_name
experiment_id = subprocess.getoutput(f"nvtl {model_name} experiment-create --network_arch {network_arch} --encryption_key {encode_key} ")
print(experiment_id)

### Assign train, eval datasets <a class="anchor" id="head-10"></a>

In [ ]:
docker_env_vars = {} # Update any variables to be included while triggering Docker run-time like MLOPs variables 
dataset_information = {}
dataset_information["train_datasets"] = [train_dataset_id]
if model_name in ("bpnet","fpenet","lprnet","ml_recog","ocdnet","ocrnet"):
    dataset_information["calibration_dataset"] = train_dataset_id
if model_name in ("lprnet", "ocdnet", "ocrnet", "optical_inspection") or ds_format == "visual_changenet_classify":
    dataset_information["eval_dataset"] = eval_dataset_id
if model_name in ("lprnet", "optical_inspection") or ds_format == "visual_changenet_classify":
    dataset_information["inference_dataset"] = test_dataset_id
if model_name in ("centerpose"):
    dataset_information["eval_dataset"] = train_dataset_id
    dataset_information["inference_dataset"] = train_dataset_id
dataset_information["docker_env_vars"] = docker_env_vars
patched_model = subprocess.getoutput(f"nvtl {model_name} patch-artifact-metadata --id {experiment_id} --job_type experiment --update_info '{json.dumps(dataset_information)}' ")
print(patched_model)

### List experiments <a class="anchor" id="head-5"></a>

In [ ]:
# List all pretrained models for the chosen network architecture
filter_params = {"network_arch": network_arch}
message = subprocess.getoutput(f"nvtl {model_name} list-experiments --filter_params '{json.dumps(filter_params)}'")
message = ast.literal_eval(message)
for rsp in message:
    rsp_keys = rsp.keys()
    if "encryption_key" not in rsp.keys():
        assert "name" in rsp_keys and "version" in rsp_keys and "ngc_path" in rsp_keys and "additional_id_info" in rsp_keys
        print(f'PTM Name: {rsp["name"]}; PTM version: {rsp["version"]}; NGC PATH: {rsp["ngc_path"]}; Additional info: {rsp["additional_id_info"]}')

### Assign PTM <a class="anchor" id="head-7"></a>

Search for PTM on NGC for the Segmentation model chosen

In [ ]:
# Assigning pretrained models to different purpose built models versions
# From the output of previous cell make the appropriate changes to this map if you want to change the default PTM backbone.
# Changing the default backbone here requires changing default spec/config during train/eval etc like for example
# If you are changing the ptm to resnet34, then you have to modify the config key num_layers if it exists to 34 manually
visual_changenet_ptm = "visual_changenet_segmentation_levircd:visual_changenet_levircd_trainable_v1.0" # For segmentation
if model_name == "visual_changenet" and ds_format == 'visual_changenet_classify':
    visual_changenet_ptm = "visual_changenet_classification:visual_changenet_nvpcb_trainable_v1.0"
pretrained_map = {"action_recognition":"actionrecognitionnet:trainable_v1.0",
                  "bpnet" : "bodyposenet:trainable_v1.0",
                  "fpenet" : "fpenet:trainable_v1.0",
                  "lprnet": "lprnet:trainable_v1.0",
                  "ml_recog": "retail_object_recognition:trainable_v1.0",
                  "ocdnet": "ocdnet:trainable_resnet18_v1.0",
                  "ocrnet": "ocrnet:trainable_v1.0",
                  "optical_inspection": "optical_inspection:trainable_v1.0",
                  "pointpillars":"pointpillarnet:trainable_v1.0",
                  "pose_classification":"poseclassificationnet:trainable_v1.0",
                  "re_identification":"reidentificationnet:trainable_v1.1",
                  "visual_changenet":visual_changenet_ptm,
                  "centerpose": "pretrained_fan_classification_nvimagenet:fan_small_hybrid_nvimagenet"}

if model_name == "action_recognition":
    if model_type == "of":
        pretrained_map["action_recognition"] = "actionrecognitionnet:trainable_v2.0"
    elif model_type == "joint":
        pretrained_map["action_recognition"] = "actionrecognitionnet:trainable_v1.0,actionrecognitionnet:trainable_v2.0"

no_ptm_models = set([])

In [ ]:
if network_arch not in no_ptm_models:
    filter_params = {"network_arch": network_arch}
    message = subprocess.getoutput(f"nvtl {model_name} list-experiments --filter_params '{json.dumps(filter_params)}'")
    message = ast.literal_eval(message)

    ptm_model_names = pretrained_map[network_arch].split(",")
    ptm = []

    # Search for ptm with given ngc path
    for ptm_model_name in ptm_model_names:
        ptm_id = None
        for rsp in message:
            rsp_keys = rsp.keys()
            assert "ngc_path" in rsp_keys
            if rsp["ngc_path"].endswith(pretrained_map[network_arch]):
                additional_id_info = []
                if rsp["additional_id_info"]:
                    assert "additional_id_info" in rsp_keys
                    additional_id_info = rsp["additional_id_info"].split(",")
                if (len(additional_id_info) == 0) or \
                    (network_arch == "lprnet" and len(additional_id_info) == 1 and additional_id_info[0] == model_type) or \
                    (network_arch == "action_recognition" and len(additional_id_info) == 1 and additional_id_info[0] == model_input_type) or \
                    (network_arch == "action_recognition" and len(additional_id_info) == 2 and additional_id_info[0] == platform and additional_id_info[1] == model_input_type):
                    assert "id" in rsp_keys
                    ptm_id = rsp["id"]
                    print("Metadata for model with requested NGC Path")
                    print(rsp)
                    break
        ptm.append(ptm_id)


In [ ]:
if network_arch not in no_ptm_models:
    ptm_information = {"base_experiment":ptm}
    patched_model = subprocess.getoutput(f"nvtl {model_name} patch-artifact-metadata --id {experiment_id} --job_type experiment --update_info '{json.dumps(ptm_information)}' ")
    print(patched_model)

### View hyperparameters that are enabled for AutoML by default <a class="anchor" id="head-13"></a>

In [ ]:
if automl_enabled:
    # View default automl specs enabled
    ! nvtl {model_name} model-automl-defaults --id {experiment_id}

### Train <a class="anchor" id="head-11"></a>

#### Set AutoML related configurations <a class="anchor" id="head-14"></a>
Refer to these hyper-links to see the parameters supported by each network and add more parameters if necessary in addition to the default automl enabled parameters:

[ActionRecognitionNet](https://github.com/NVIDIA/tao_front_end_services/tree/main/api/specs_utils/specs/action_recognition/action_recognition%20-%20train.csv), 
[BPNET](https://github.com/NVIDIA/tao_front_end_services/tree/main/api/specs_utils/specs/bpet/bpnet%20-%20train.csv), 
[FPENET](https://github.com/NVIDIA/tao_front_end_services/tree/main/api/specs_utils/specs/fpenet/fpenet%20-%20train.csv), 
[LPRNET](https://github.com/NVIDIA/tao_front_end_services/tree/main/api/specs_utils/specs/lprnet/lprnet%20-%20train.csv), 
[MetricLearningRecognition](https://github.com/NVIDIA/tao_front_end_services/tree/main/api/specs_utils/specs/ml_recog/ml_recog%20-%20train.csv), 
[OCDNET](https://github.com/NVIDIA/tao_front_end_services/tree/main/api/specs_utils/specs/ocdnet/ocdnet%20-%20train.csv), 
[OCRNET](https://github.com/NVIDIA/tao_front_end_services/tree/main/api/specs_utils/specs/ocrnet/ocrnet%20-%20train.csv), 
[OpticalInspection](https://github.com/NVIDIA/tao_front_end_services/tree/main/api/specs_utils/specs/optical_inspection/optical_inspection%20-%20train.csv), 
[Pointpillars](https://github.com/NVIDIA/tao_front_end_services/tree/main/api/specs_utils/specs/pointpillars/pointpillars%20-%20train.csv), 
[PoseClassificationNet](https://github.com/NVIDIA/tao_front_end_services/tree/main/api/specs_utils/specs/pose_classification/pose_classification%20-%20train.csv), 
[ReIdentificationNet](https://github.com/NVIDIA/tao_front_end_services/tree/main/api/specs_utils/specs/re_identification/re_identification%20-%20train.csv)

In [ ]:
if automl_enabled:
    # Choose any metric that is present in the kpi dictionary present in the model's status.json. 
    # Example status.json for each model can be found in the respective section in NVIDIA TAO DOCS here: https://docs.nvidia.com/tao/tao-toolkit/text/model_zoo/cv_models/index.html
    metric="kpi"

    additional_automl_parameters = [] #Refer to parameter list mentioned in the above links and add any extra parameter in addition to the default enabled ones
    remove_default_automl_parameters = [] #Remove any hyperparameters that are enabled by default for AutoML

    automl_information = {"automl_enabled":automl_enabled,
                          "automl_algorithm":automl_algorithm,
                          "automl_max_recommendations": 20, # Only for bayesian
                          "automl_R": 27, # Only for hyperband
                          "automl_nu": 3, # Only for hyperband
                          "epoch_multiplier": 1, # Only for hyperband
                          # Enable this if you want to add parameters to automl_add_hyperparameters below that are disabled by TAO in the automl_enabled column of the spec csv.
                          # Warning: The parameters that are disabled are not tested by TAO, so there might be unexpected behaviour in overriding this
                          "override_automl_disabled_params": False,
                          "metric":metric,
                          "automl_add_hyperparameters":str(additional_automl_parameters),
                          "automl_remove_hyperparameters":str(remove_default_automl_parameters)
                         }
    patched_model = subprocess.getoutput(f"nvtl {model_name} patch-artifact-metadata --id {experiment_id} --job_type experiment --update_info '{json.dumps(automl_information)}' ")
    patched_model = json.loads(patched_model)
    print(json.dumps(patched_model, indent=4))

#### Provide train specs <a class="anchor" id="head-15"></a>

In [ ]:
# Default train model specs
train_specs = subprocess.getoutput(f"nvtl {model_name} get-spec --action train --job_type experiment --id {experiment_id}")
train_specs = json.loads(train_specs)
print(json.dumps(train_specs, indent=4))

In [ ]:
# Apply changes for any of the parameters listed in the previous cell as required
if model_name == "action_recognition":
    train_specs["model"]["model_type"] = model_type
    train_specs["model"]["input_type"] = model_input_type
    train_specs["train"]["num_epochs"] = 20
    train_specs["train"]["num_gpus"] = 1
elif model_name == "bpnet":
    train_specs["num_epoch"] = 20
    train_specs["checkpoint_n_epoch"] = 5
    train_specs["validation_every_n_epoch"] = 5
    train_specs["finetuning_config"]["checkpoint_path"] = None
    train_specs["gpus"] = 1
elif model_name == "centerpose":
    train_specs["train"]["num_epochs"] = 20 # Please set it to 140 if you want to run the whole training.
    train_specs["train"]["validation_interval"] = 10
    train_specs["train"]["checkpoint_interval"] = 10
    train_specs["train"]["num_gpus"] = 1
    train_specs["dataset"]["category"] = testing_categories
elif model_name == "fpenet":
    train_specs["num_epoch"] = 10
    train_specs["checkpoint_n_epoch"] = 5
    train_specs["dataloader"]["dataset_info"]["root_path"] = None
    train_specs["num_keypoints"] = int(model_type)
    train_specs["dataloader"]["num_keypoints"] = int(model_type)
    train_specs["gpus"] = 1
elif model_name == "lprnet":
    train_specs["training_config"]["num_epochs"] = 24
    train_specs["gpus"] = 1
elif model_name == "ml_recog":
    train_specs["train"]["num_epochs"] = 30
    train_specs["train"]["gpu_ids"] = [0]
    train_specs["train"]["checkpoint_interval"] = 5
elif model_name == "ocdnet":
    train_specs["train"]["num_epochs"] = 30
    train_specs["train"]["checkpoint_interval"] = 5
    train_specs["train"]["validation_interval"] = 5
    train_specs["train"]["gpu_id"] = [0]
    train_specs["num_gpus"] = 1
elif model_name == "ocrnet":
    train_specs["train"]["num_epochs"] = 20
    train_specs["train"]["checkpoint_interval"] = 5
    train_specs["train"]["validation_interval"] = 5
    train_specs["train"]["num_gpus"] = 1
elif model_name == "optical_inspection":
    train_specs["train"]["num_epochs"] = 20
    train_specs["train"]["checkpoint_interval"] = 5
    train_specs["train"]["validation_interval"] = 5
    train_specs["train"]["gpu_ids"] = [0]
elif model_name == "pose_classification":
    train_specs["train"]["num_epochs"] = 50
    train_specs["train"]["gpu_ids"] = [0]
    train_specs["train"]["num_gpus"] = 1
    if model_type == "nvidia":
        train_specs["dataset"]["num_classes"] = 6
        train_specs["model"]["graph_layout"] = "nvidia"
    elif model_type == "kinetics":
        train_specs["dataset"]["num_classes"] = 5
        train_specs["model"]["graph_layout"] = "openpose"
elif model_name == "pointpillars":
    train_specs["train"]["num_epochs"] = 80
    train_specs["gpus"] = 1
elif model_name == "re_identification":
    train_specs["train"]["num_epochs"] = 120
    train_specs["train"]["gpu_ids"] = [0]
    train_specs["train"]["num_gpus"] = 1
    train_specs["dataset"]["num_classes"] = 100 #The number set in obtain_subset script
    train_specs["dataset"]["num_workers"] = 4 #Modify the num_workers according to your hardware setup
    train_specs["dataset"]["batch_size"] = 16 #Modify the batch_size according to your hardware setup
elif model_name == "visual_changenet":
    train_specs["train"]["num_epochs"] = 30 
    train_specs["train"]["checkpoint_interval"] = 2
    train_specs["train"]["val_interval"] = 5
    train_specs["num_gpus"] = 1
    if ds_format == "visual_changenet_segment":
        train_specs["task"] = 'segment'
    elif ds_format == "visual_changenet_classify":
        train_specs["task"] = 'classify'
print(json.dumps(train_specs, indent=4))

#### Run train <a class="anchor" id="head-16"></a>

In [ ]:
ds_convert_parent = ""
if model_name in ("bpnet", "ocrnet"):
    val_convert_job_id = job_map["eval_convert_" + model_name]
    ds_convert_parent = f"--parent_job_id {val_convert_job_id}"
elif model_name in ("bpnet", "fpenet", "ocrnet", "pointpillars"):
    train_convert_job_id = job_map["train_convert_" + model_name]
    ds_convert_parent = f"--parent_job_id {train_convert_job_id}"

job_id = subprocess.getoutput(f"nvtl {model_name} experiment-run-action --action train --id {experiment_id} --specs '{json.dumps(train_specs)}' {ds_convert_parent}")
if model_name == "visual_changenet":
    job_map["train_" + ds_format] = job_id
else:
    job_map["train_" + model_name] = job_id
print(job_id)

In [ ]:
# Monitor job status
if automl_enabled:    
    while True:
        clear_output(wait=True)
        response = subprocess.getoutput(f"nvtl {model_name} get-action-status --job_type experiment --id {experiment_id} --job {job_id}")
        response = json.loads(response)
        if "error_desc" in response.keys() and response["error_desc"] in ("Job trying to retrieve not found", "No AutoML run found"):
            print("Job is being created")
            time.sleep(5)
            continue
        print(json.dumps(response, sort_keys=True, indent=4))
        assert "status" in response.keys() and response.get("status") != "Error"
        if response.get("status") in ["Done","Error"]:
            break
        time.sleep(15)
else:
    # Check status (the file won't exist until the backend Toolkit container is running -- can take several minutes)
    status = my_tail(model_name, experiment_id, job_id, "experiment", workdir)

In [ ]:
## To Stop an AutoML JOB
#    1. Stop the 'Monitor job status' cell (the cell right before this cell) manually
#    2. Uncomment the snippet in the next cell and run the cell

In [ ]:
# if automl_enabled:
#     if model_name == "visual_changenet":
#         job_id = job_map["train_" + ds_format]
#     else:
#         job_id = job_map["train_" + model_name]
#     job_id = subprocess.getoutput(f"nvtl {model_name} job-cancel --job_type experiment --id {experiment_id} --job {job_id}")
#     if model_name == "visual_changenet":
#         job_map["canceled_" + ds_format] = job_id
#     else:
#         job_map["canceled_" + model_name] = job_id
#     print(job_id)

In [ ]:
## Resume AutoML

In [ ]:
# Uncomment the below snippet if you want to resume an already stopped AutoML job and then run the 'Monitor job status' cell above (4th cell above from this cell)
# if automl_enabled:
#     if model_name == "visual_changenet":
#         job_id = job_map["train_" + ds_format]
#     else:
#         job_id = job_map["train_" + model_name]
#     job_id = subprocess.getoutput(f"nvtl {model_name} job-resume --id {experiment_id} --job {job_id} --specs '{json.dumps(train_specs)}' {ds_convert_parent}")
#     if model_name == "visual_changenet":
#         job_map["resumed_" + ds_format] = job_id
#     else:
#         job_map["resumed_" + model_name] = job_id
#     print(job_id)

### Download train job artifacts <a class="anchor" id="head-17"></a>

In [ ]:
if model_name == "visual_changenet":
    job_id = job_map["train_" + ds_format]
else:
    job_id = job_map["train_" + model_name]
file_list = subprocess.getoutput(f"nvtl {model_name} list-job-files --id {experiment_id} --job {job_id} --job_type experiment --retrieve_logs True --retrieve_specs False")
print(file_list)

In [ ]:
## Patch the model with proper metric before training to run this cell; By default loss is used, but some models dont log the parameter under the name 'loss'
# file_lists = []
# temptar = subprocess.getoutput(f"nvtl {model_name} download-selective-files --id {experiment_id} --job {job_id} --job_type experiment --workdir {workdir}  --file_lists '{file_lists}' --best_model False --latest_model True --tar_files True")
# tar_command = f'tar -xvf {temptar} -C {workdir}/'
# os.system(tar_command)
# os.remove(temptar)
# print(f"Results at {workdir}/{job_id}")
# model_downloaded_path = f"{workdir}/{job_id}"

In [ ]:
# Downloading train job takes a longer time, uncomment this cell if you want to still proceed
if download_jobs:
    temptar = subprocess.getoutput(f"nvtl {model_name} download-entire-job --id {experiment_id} --job {job_id} --job_type experiment --workdir {workdir}")
    tar_command = f'tar -xvf {temptar} -C {workdir}/'
    os.system(tar_command)
    os.remove(temptar)
    print(f"Results at {workdir}/{job_id}")
    model_downloaded_path = f"{workdir}/{job_id}"

In [ ]:
# View the checkpoints generated for the training job and for automl jobs, in addition view: best performing model's config and the results of all automl experiments
if download_jobs:
    if automl_enabled:
        !python3 -m pip install pandas==1.5.1
        import pandas as pd
        model_downloaded_path = f"{model_downloaded_path}/best_model"
        assert glob.glob(f"{model_downloaded_path}/*.protobuf") or glob.glob(f"{model_downloaded_path}/*.yaml")

    assert os.path.exists(model_downloaded_path)
    assert (glob.glob(model_downloaded_path + "/**/*.tlt", recursive=True) + glob.glob(model_downloaded_path + "/**/*.hdf5", recursive=True) + glob.glob(model_downloaded_path + "/**/*.pth", recursive=True))

    if os.path.exists(model_downloaded_path):        
        #List the binary model file
        print("\nCheckpoints for the training experiment")
        if os.path.exists(model_downloaded_path+"/train/weights") and len(os.listdir(model_downloaded_path+"/train/weights")) > 0:
            print(f"Folder: {model_downloaded_path}/train/weights")
            print("Files:", os.listdir(model_downloaded_path+"/train/weights"))
        elif os.path.exists(model_downloaded_path+"/weights") and len(os.listdir(model_downloaded_path+"/weights")) > 0:
            print(f"Folder: {model_downloaded_path}/weights")
            print("Files:", os.listdir(model_downloaded_path+"/weights"))
        else:
            print(f"Folder: {model_downloaded_path}")
            print("Files:", os.listdir(model_downloaded_path))

        if automl_enabled:
            assert glob.glob(f"{model_downloaded_path}/*.protobuf") or glob.glob(f"{model_downloaded_path}/*.yaml")
            experiment_artifacts = json.load(open(f"{model_downloaded_path}/controller.json","r"))
            data_frame = pd.DataFrame(experiment_artifacts)
            # Print experiment id/number and the corresponding result
            print("\nResults of all experiments")
            with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):
                print(data_frame[["id","result"]])

### Evaluate <a class="anchor" id="head-16"></a>

#### Provide evaluate specs <a class="anchor" id="head-18"></a>

In [ ]:
# Default evaluate model specs
eval_specs = subprocess.getoutput(f"nvtl {model_name} get-spec --action evaluate --job_type experiment --id {experiment_id}")
eval_specs = json.loads(eval_specs)
print(json.dumps(eval_specs, indent=4))

In [ ]:
# Customize evaluate model specs
if model_name == "action_recognition":
    eval_specs["model"]["model_type"] = model_type
    eval_specs["model"]["input_type"] = model_input_type
elif model_name == "fpenet":
    eval_specs["dataloader"]["dataset_info"]["root_path"] = None
    eval_specs["num_keypoints"] = int(model_type)
    eval_specs["dataloader"]["num_keypoints"] = int(model_type)
elif model_name == "pose_classification":
    if model_type == "nvidia":
        eval_specs["dataset"]["num_classes"] = 6
        eval_specs["model"]["graph_layout"] = "nvidia"
    elif model_type == "kinetics":
        eval_specs["dataset"]["num_classes"] = 5
        eval_specs["model"]["graph_layout"] = "openpose"
elif model_name == "re_identification":
    eval_specs["dataset"]["num_classes"] = 100 #The number set in obtain_subset script
elif ds_format == 'visual_changenet_segment':
    eval_specs["task"] = 'segment'
elif ds_format == 'visual_changenet_classify':
    eval_specs["task"] = 'classify'
    eval_specs["train"]["classify"]["loss"] = "contrastive"
elif model_name == "centerpose":
    eval_specs["dataset"]["category"] = testing_categories
print(json.dumps(eval_specs, indent=4))

#### Run evaluate <a class="anchor" id="head-19"></a>

In [ ]:
# Print model handler parameters
model_parameters = subprocess.getoutput(f"nvtl {model_name} get-metadata --id {experiment_id} --job_type experiment")
model_parameters = json.loads(model_parameters)
update_checkpoint_choosing = {}
update_checkpoint_choosing["checkpoint_choose_method"] = model_parameters["checkpoint_choose_method"]
update_checkpoint_choosing["checkpoint_epoch_number"] = model_parameters["checkpoint_epoch_number"]
print(json.dumps(update_checkpoint_choosing, indent=4))

In [ ]:
# Change the method by which checkpoint from the parent action is chosen, when parent action is a train/retrain action.
# Example for evaluate action below, can be applied in the same way for other actions too
update_checkpoint_choosing["checkpoint_choose_method"] = "latest_model" # Choose between best_model/latest_model/from_epoch_number
# If from_epoch_number is chosen then assign the epoch number to the dictionary key in the format 'from_epoch_number{train_job_id}'
# update_checkpoint_choosing["checkpoint_epoch_number"]["from_epoch_number_c2f76eb7-2a75-4197-9a84-c1547f20c17d"] = 6

patched_model = subprocess.getoutput(f"nvtl {model_name} patch-artifact-metadata --id {experiment_id} --job_type experiment --update_info '{json.dumps(update_checkpoint_choosing)}' ")
patched_model = json.loads(patched_model)
print(json.dumps(patched_model, indent=4))

In [ ]:
if model_name == "visual_changenet":
    parent = job_map["train_" + ds_format]
else:
    parent = job_map["train_" + model_name]
job_id = subprocess.getoutput(f"nvtl {model_name} experiment-run-action --action evaluate --id {experiment_id} --parent_job_id {parent} --specs '{json.dumps(eval_specs)}'")
if model_name == "visual_changenet":
    job_map["eval_" + ds_format] = job_id
else:
    job_map["eval_" + model_name] = job_id
print(job_id)

In [ ]:
# Check status (the file won't exist until the backend Toolkit container is running -- can take several minutes)
status = my_tail(model_name, experiment_id, job_id, "experiment", workdir)

### Prune, Retrain and Evaluation <a class="anchor" id="head-13"></a>

- We optimize the trained model by pruning and retraining in the following cells

#### Prune <a class="anchor" id="head-20"></a>

In [ ]:
# Default prune model specs
if model_name in ("bpnet", "ocdnet", "ocrnet", "pointpillars"):
    prune_specs = subprocess.getoutput(f"nvtl {model_name} get-spec --id {experiment_id} --action prune --job_type experiment")
    prune_specs = json.loads(prune_specs)
    print(json.dumps(prune_specs, indent=4))

In [ ]:
# Apply changes to prune specs if neccessary
if model_name in ("bpnet", "ocdnet", "ocrnet", "pointpillars"):
    print(json.dumps(prune_specs, indent=4))

In [ ]:
if model_name in ("bpnet","pointpillars"):
    parent = job_map["train_" + model_name]
    job_id = subprocess.getoutput(f"nvtl {model_name} experiment-run-action --action prune --id {experiment_id} --parent_job_id {parent} --specs '{json.dumps(prune_specs)}'")
    job_map["prune_" + model_name] = job_id

In [ ]:
# Check status of pruning job (the file won't exist until the backend Toolkit container is running -- can take several minutes)
if model_name in ("bpnet","pointpillars"):
    status = my_tail(model_name, experiment_id, job_id, "experiment", workdir)

#### Retrain <a class="anchor" id="head-22"></a>

In [ ]:
# Default retrain model specs
if model_name in ("bpnet", "ocdnet", "ocrnet", "pointpillars"):
    retrain_specs = subprocess.getoutput(f"nvtl {model_name} get-spec --id {experiment_id} --action retrain --job_type experiment")
    retrain_specs = json.loads(retrain_specs)
    print(json.dumps(retrain_specs, indent=4))

In [ ]:
# Customize retrain model specs
if model_name in ("bpnet", "ocdnet", "ocrnet", "pointpillars"):
    if model_name == "bpnet":
        retrain_specs["finetuning_config"]["checkpoint_path"] = None
        retrain_specs["gpus"] = 1
    elif model_name == "ocdnet":
        retrain_specs["train"]["num_epochs"] = 30
        retrain_specs["train"]["gpu_id"] = [0]
        retrain_specs["num_gpus"] = 1
    elif model_name == "ocrnet":
        retrain_specs["train"]["num_epochs"] = 20
        retrain_specs["train"]["gpu_ids"] = [0]
    elif model_name == "pointpillars":
        retrain_specs["train"]["num_epochs"] = 80
        retrain_specs["gpus"] = 1
    print(json.dumps(retrain_specs, indent=4))

In [ ]:
if model_name in ("bpnet","pointpillars"):
    parent = job_map["prune_" + model_name]
    job_id = subprocess.getoutput(f"nvtl {model_name} experiment-run-action --action retrain --id {experiment_id} --parent_job_id {parent} --specs '{json.dumps(retrain_specs)}'")
    job_map["retrain_" + model_name] = job_id

In [ ]:
# Check status of retrain job (the file won't exist until the backend Toolkit container is running -- can take several minutes)
if model_name in ("bpnet","pointpillars"):
    status = my_tail(model_name, experiment_id, job_id, "experiment", workdir)

#### Evaluate after retrain <a class="anchor" id="head-18"></a>

In [ ]:
# Default evaluate model specs
if model_name in ("bpnet", "ocdnet", "ocrnet", "pointpillars"):
    eval_retrain_specs = subprocess.getoutput(f"nvtl {model_name} get-spec --action evaluate --job_type experiment --id {experiment_id}")
    eval_retrain_specs = json.loads(eval_retrain_specs)
    print(json.dumps(eval_retrain_specs, indent=4))

In [ ]:
# Customize evaluate model specs if necessary
if model_name in ("bpnet", "ocdnet", "ocrnet", "pointpillars"):
    print(json.dumps(eval_retrain_specs, indent=4))

In [ ]:
if model_name in ("bpnet","pointpillars"):
    parent = job_map["retrain_" + model_name]
    job_id = subprocess.getoutput(f"nvtl {model_name} experiment-run-action --action evaluate --id {experiment_id} --parent_job_id {parent} --specs '{json.dumps(eval_retrain_specs)}'")
    job_map["eval2_" + model_name] = job_id

In [ ]:
# Check status (the file won't exist until the backend Toolkit container is running -- can take several minutes)
if model_name in ("bpnet","pointpillars"):
    status = my_tail(model_name, experiment_id, job_id, "experiment", workdir)

### Export <a class="anchor" id="head-22"></a>

#### Provide Export specs <a class="anchor" id="head-25"></a>

In [ ]:
# Default export model specs
export_specs = subprocess.getoutput(f"nvtl {model_name} get-spec --id {experiment_id} --action export --job_type experiment")
export_specs = json.loads(export_specs)
print(json.dumps(export_specs, indent=4))

In [ ]:
# Customize export model specs
if model_name == "action_recognition":
    export_specs["model"]["model_type"] = model_type
    export_specs["model"]["input_type"] = model_input_type
elif model_name == "lprnet":
    export_specs["data_type"] = "fp32"
elif model_name == "bpnet":
    export_specs["data_type"] = "int8"
elif model_name == "pose_classification":
    if model_type == "nvidia":
        export_specs["dataset"]["num_classes"] = 6
        export_specs["model"]["graph_layout"] = "nvidia"
    elif model_type == "kinetics":
        export_specs["dataset"]["num_classes"] = 5
        export_specs["model"]["graph_layout"] = "openpose"
elif model_name == "re_identification":
    export_specs["dataset"]["num_classes"] = 100 #The number set in obtain_subset script
elif ds_format == 'visual_changenet_segment':
    export_specs["export"]["input_height"] = 256 
    export_specs["export"]["input_width"] = 256 
    export_specs["task"] = 'segment'
elif ds_format == 'visual_changenet_classify':
    export_specs["export"]["input_height"] = 512 
    export_specs["export"]["input_width"] = 128
    export_specs["task"] = 'classify'
print(json.dumps(export_specs, indent=4))

#### Run export <a class="anchor" id="head-26"></a>

In [ ]:
if model_name == "visual_changenet":
    parent = job_map["train_" + ds_format]
else:
    parent = job_map["train_" + model_name]
job_id = subprocess.getoutput(f"nvtl {model_name} experiment-run-action --action export --id {experiment_id} --parent_job_id {parent} --specs '{json.dumps(export_specs)}'")
if model_name == "visual_changenet":
    job_map["export_" + ds_format] = job_id
else:
    job_map["export_" + model_name] = job_id
print(job_id)

In [ ]:
# Check status (the file won't exist until the backend Toolkit container is running -- can take several minutes)
status = my_tail(model_name, experiment_id, job_id, "experiment", workdir)

### TRT Engine generation using TAO-Deploy <a class="anchor" id="head-26"></a>

#### Provide trt engine generation specs <a class="anchor" id="head-27"></a>

In [ ]:
# Default gen_trt_engine model specs
if model_name in ("bpnet", "lprnet", "ocdnet", "ocrnet", "ml_recog","optical_inspection", "visual_changenet", "centerpose"):
    if model_name in ("lprnet", "ocdnet", "ocrnet", "ml_recog","optical_inspection", "visual_changenet", "centerpose"):
        tao_deploy_specs = subprocess.getoutput(f"nvtl {model_name} get-spec --id {experiment_id} --action gen_trt_engine --job_type experiment")
    elif model_name in ("bpnet"):
        tao_deploy_specs = subprocess.getoutput(f"nvtl {model_name} get-spec --id {experiment_id} --action trtexec --job_type experiment")
    tao_deploy_specs = json.loads(tao_deploy_specs)
    print(json.dumps(tao_deploy_specs, indent=4))

In [ ]:
# Customize convert model specs
if model_name in ("bpnet", "lprnet", "ocdnet", "ocrnet", "ml_recog","optical_inspection", "visual_changenet"):
    # Make changes to the specs dictionary if required here
    if model_name == "lprnet":
        tao_deploy_specs["data_type"] = "fp32"
    elif model_name in ("ml_recog", "ocdnet"):
        tao_deploy_specs["gen_trt_engine"]["tensorrt"]["data_type"] = "int8"
    elif model_name in ("ocrnet", "optical_inspection"):
        tao_deploy_specs["gen_trt_engine"]["tensorrt"]["data_type"] = "fp16"   
    elif ds_format == 'visual_changenet_classify':
        tao_deploy_specs["gen_trt_engine"]["input_height"] = 512 
        tao_deploy_specs["gen_trt_engine"]["input_width"] = 128
        tao_deploy_specs["task"] = 'classify'
    elif ds_format == 'visual_changenet_segment':
        tao_deploy_specs["gen_trt_engine"]["tensorrt"]["data_type"] = "fp16"
        tao_deploy_specs["gen_trt_engine"]["input_height"] = 256
        tao_deploy_specs["gen_trt_engine"]["input_width"]= 256
        tao_deploy_specs["task"] = 'segment' 
    print(json.dumps(tao_deploy_specs, indent=4))

#### Run TRT engine generation using TAO-Deploy <a class="anchor" id="head-28"></a>

In [ ]:
if model_name in ("bpnet", "lprnet", "ocdnet", "ocrnet", "ml_recog","optical_inspection", "visual_changenet", "centerpose"):
    if model_name == "visual_changenet":
        parent = job_map["export_" + ds_format]
    else:
        parent = job_map["export_" + model_name]
    if model_name in ("lprnet", "ocdnet", "ocrnet", "ml_recog","optical_inspection", "visual_changenet", "centerpose"):
        job_id = subprocess.getoutput(f"nvtl {model_name} experiment-run-action --action gen_trt_engine --id {experiment_id} --parent_job_id {parent} --specs '{json.dumps(tao_deploy_specs)}'")
    elif model_name in ("bpnet"):
        job_id = subprocess.getoutput(f"nvtl {model_name} experiment-run-action --action trtexec --id {experiment_id} --parent_job_id {parent} --specs '{json.dumps(tao_deploy_specs)}'")
    if model_name == "visual_changenet":
        job_map["gen_trt_engine_" + ds_format] = job_id
    else:
        job_map["gen_trt_engine_" + model_name] = job_id
    print(job_id)

In [ ]:
# Check status (the file won't exist until the backend Toolkit container is running -- can take several minutes)
if model_name in ("bpnet", "lprnet", "ocdnet", "ocrnet", "ml_recog","optical_inspection", "visual_changenet", "centerpose"):
    status = my_tail(model_name, experiment_id, job_id, "experiment", workdir)

### TAO inference <a class="anchor" id="head-28"></a>

#### Provide TAO inference specs <a class="anchor" id="head-29"></a>

In [ ]:
# Default inference model specs
tao_inference_specs = subprocess.getoutput(f"nvtl {model_name} get-spec --id {experiment_id} --action inference --job_type experiment")
tao_inference_specs = json.loads(tao_inference_specs)
print(json.dumps(tao_inference_specs, indent=4))

In [ ]:
# Customize TAO inference specs
if model_name == "action_recognition":
    tao_inference_specs["model"]["model_type"] = model_type
    tao_inference_specs["model"]["input_type"] = model_input_type
elif model_name == "fpenet":
    tao_inference_specs["num_keypoints"] = int(model_type)
    tao_inference_specs["dataloader"]["num_keypoints"] = int(model_type)
elif model_name == "pose_classification":
    if model_type == "nvidia":
        tao_inference_specs["dataset"]["num_classes"] = 6
        tao_inference_specs["model"]["graph_layout"] = "nvidia"
    elif model_type == "kinetics":
        tao_inference_specs["dataset"]["num_classes"] = 5
        tao_inference_specs["model"]["graph_layout"] = "openpose"
elif model_name == "re_identification":
    tao_inference_specs["dataset"]["num_classes"] = 100 #The number set in obtain_subset script
elif ds_format == 'visual_changenet_classify':
    tao_inference_specs["inference"]["batch_size"] = tao_inference_specs["dataset"]["classify"]['batch_size'] 
    tao_inference_specs["task"] = 'classify'
elif ds_format == 'visual_changenet_segment':
    tao_inference_specs["inference"]["batch_size"] = tao_inference_specs["dataset"]["segment"]['batch_size'] 
    tao_inference_specs["task"] = 'segment'
elif model_name == "centerpose":
    tao_inference_specs["dataset"]["category"] = testing_categories
print(json.dumps(tao_inference_specs, indent=4))

#### Run TAO inference <a class="anchor" id="head-30"></a>

In [ ]:
if model_name == "visual_changenet":
    parent = job_map["train_" + ds_format]
else:
    parent = job_map["train_" + model_name]
job_id = subprocess.getoutput(f"nvtl {model_name} experiment-run-action --action inference --id {experiment_id} --parent_job_id {parent} --specs '{json.dumps(tao_inference_specs)}'")
if model_name == "visual_changenet":
    job_map["tlt_inference_" + ds_format] = job_id
else:
    job_map["tlt_inference_" + model_name] = job_id
print(job_id)

In [ ]:
# Check status (the file won't exist until the backend Toolkit container is running -- can take several minutes)
status = my_tail(model_name, experiment_id, job_id, "experiment", workdir)

In [ ]:
if download_jobs:
    temptar = subprocess.getoutput(f"nvtl {model_name} download-entire-job --id {experiment_id} --job {job_id} --job_type experiment --workdir {workdir}")
    tar_command = f'tar -xvf {temptar} -C {workdir}/'
    os.system(tar_command)
    os.remove(temptar)
    print(f"Results at {workdir}/{job_id}")
    inference_out_path = f"{workdir}/{job_id}"

In [ ]:
# Inference output must be here
if download_jobs:
    if model_name in ("action_recognition","lprnet","ocrnet"):
        assert os.path.exists(f'{inference_out_path}/logs_from_toolkit.txt')
        !cat {inference_out_path}/logs_from_toolkit.txt
    elif model_name in ("bpnet","pointpillars"):
        if model_name == "bpnet":
            assert glob.glob(f"{inference_out_path}/images_annotated/*.png")
        elif model_name == "pointpillars":
            assert glob.glob(f"{inference_out_path}/infer/detected_boxes/*.png")
        elif model_name == "centerpose":
            assert glob.glob(f"{inference_out_path}/inference/*.png")
        !python3 -m pip install matplotlib
        import matplotlib.pyplot as plt
        import matplotlib.image as mpimg
        if model_name == "bpnet":
            sample_image = glob.glob(f"{inference_out_path}/images_annotated/*.png")[0]
        elif model_name == "pointpillars":
            sample_image = glob.glob(f"{inference_out_path}/infer/detected_boxes/*.png")[0]
        elif model_name == "centerpose":
            sample_image = glob.glob(f"{inference_out_path}/inference/*.png")[0]
        def display_photo(path):
            img = mpimg.imread(path)
            plt.figure(figsize = (int(img.shape[0]/100)*2,int(img.shape[1]/100)*2))
            plt.axis('off')
            imgplot = plt.imshow(img, aspect='auto')
            plt.show()
        display_photo(sample_image)
    elif model_name == "fpenet":
        assert os.path.exists(f'{inference_out_path}/result.txt')
        !cat {inference_out_path}/result.txt
    elif model_name == "ml_recog":
        assert os.path.exists(f'{inference_out_path}/inference/result.csv')
        !cat {inference_out_path}/inference/result.csv
    elif model_name == "optical_inspection":
        assert os.path.exists(f'{inference_out_path}/inference/inference.csv')
        !cat {inference_out_path}/inference/inference.csv
    elif model_name == "pose_classification":
        assert os.path.exists(f'{inference_out_path}/results.txt')
        !cat {inference_out_path}/results.txt
    elif model_name == "re_identification":
        assert os.path.exists(f'{inference_out_path}/inference.json')
        !cat {inference_out_path}/inference.json
    elif model_name == "visual_changenet":
        if ds_format == 'visual_changenet_classify':
            assert os.path.exists(f'{inference_out_path}/inference/inference.csv')
            !cat {inference_out_path}/inference/inference.csv
        elif ds_format == 'visual_changenet_segment':
            assert os.path.exists(f'{inference_out_path}/inference/status.json')
            !cat {inference_out_path}/inference/status.json

### TRT inference <a class="anchor" id="head-30"></a>

#### Provide TRT inference specs <a class="anchor" id="head-31"></a>

In [ ]:
# Default inference model specs
if model_name in ("bpnet", "lprnet", "ocdnet", "ocrnet", "ml_recog","optical_inspection", "visual_changenet", "centerpose"):
    trt_inference_specs = subprocess.getoutput(f"nvtl {model_name} get-spec --id {experiment_id} --action inference --job_type experiment")
    trt_inference_specs = json.loads(trt_inference_specs)
    print(json.dumps(trt_inference_specs, indent=4))

In [ ]:
# Customize TRT inference specs
# Change any spec if you wish
if model_name in ("bpnet", "lprnet", "ocdnet", "ocrnet", "ml_recog","optical_inspection", "visual_changenet", "centerpose"):
    if ds_format == 'visual_changenet_classify':
        trt_inference_specs["inference"]["batch_size"] = trt_inference_specs["dataset"]["classify"]['batch_size']
        trt_inference_specs["task"] = 'classify'
    elif ds_format == 'visual_changenet_segment':
        trt_inference_specs["inference"]["batch_size"] = trt_inference_specs["dataset"]["segment"]['batch_size']
        trt_inference_specs["task"] = 'segment'
    print(json.dumps(trt_inference_specs, indent=4))

#### Run TRT inference <a class="anchor" id="head-32"></a>

In [ ]:
if model_name in ("bpnet", "lprnet", "ocdnet", "ocrnet", "ml_recog","optical_inspection", "visual_changenet", "centerpose"):
    if model_name == "visual_changenet":
        parent = job_map["gen_trt_engine_" + ds_format]
    else:
        parent = job_map["gen_trt_engine_" + model_name]
    job_id = subprocess.getoutput(f"nvtl {model_name} experiment-run-action --action inference --id {experiment_id} --parent_job_id {parent} --specs '{json.dumps(trt_inference_specs)}'")
    if model_name == "visual_changenet":
        job_map["trt_inference_" + ds_format] = job_id
    else:
        job_map["trt_inference_" + model_name] = job_id
    print(job_id)

In [ ]:
# Check status (the file won't exist until the backend Toolkit container is running -- can take several minutes)
if model_name in ("bpnet", "lprnet", "ml_recog", "ocdnet", "ocrnet","optical_inspection", "visual_changenet", "centerpose"):
    status = my_tail(model_name, experiment_id, job_id, "experiment", workdir)

In [ ]:
if download_jobs:
    if model_name in ("bpnet", "lprnet", "ml_recog", "ocdnet", "ocrnet","optical_inspection", "visual_changenet", "centerpose"):
        temptar = subprocess.getoutput(f"nvtl {model_name} download-entire-job --id {experiment_id} --job {job_id} --job_type experiment --workdir {workdir}")
        tar_command = f'tar -xvf {temptar} -C {workdir}/'
        os.system(tar_command)
        os.remove(temptar)
        print(f"Results at {workdir}/{job_id}")
        job_dir = f"{workdir}/{job_id}"

In [ ]:
if download_jobs:
    if model_name in ("bpnet", "lprnet", "ml_recog", "ocdnet", "ocrnet","optical_inspection", "visual_changenet", "centerpose"):
        # You can view the predictions here or in the subdirectories
        !ls {job_dir}/

### Delete model <a class="anchor" id="head-21"></a>

In [ ]:
subprocess.getoutput(f"nvtl {model_name} experiment-delete --id {experiment_id}")

### Delete dataset <a class="anchor" id="head-21"></a>

#### Delete train dataset <a class="anchor" id="head-21"></a>

In [ ]:
subprocess.getoutput(f"nvtl {model_name} dataset-delete --id {train_dataset_id}")

#### Delete val dataset <a class="anchor" id="head-21"></a>

In [ ]:
if model_name in ("lprnet","ocdnet","ocrnet","optical_inspection") or ds_format == 'visual_changenet_classify':
    subprocess.getoutput(f"nvtl {model_name} dataset-delete --id {eval_dataset_id}")

#### Delete test dataset <a class="anchor" id="head-21"></a>

In [ ]:
if model_name in ("lprnet", "optical_inspection") or ds_format == 'visual_changenet_classify':
    subprocess.getoutput(f"nvtl {model_name} dataset-delete --id {test_dataset_id}")